# PyTorch Fundamentals

In this notebook, I review basic tensor operations in PyTorch and building neural networks, mostly following Chapters 12 and 13 of <i>''Machine Learning with PyTorch and Scikit-Learn''</i> by Sebastian Raschka et al. This notebook covers:

1. Creating tensors in PyTorch
2. Manipulating data type and shape of tensor
3. Operations on tensors
4. Building input pipelines
5. Simple linear regression model in PyTorch
6. XOR classification with nn.Sequential
7. XOR classification with nn.Module

In [ ]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt